# Математические основы защиты информации 
# Лабораторная работа №6
# Криптосистема RSA

In [1]:
using Primes
"""
    pow(number, power, base)
"""
function pow(number::Signed, power::Signed, base::Signed)::BigInt
    power = digits(power, base=2);
    result = BigInt(1);
    for i = length(power):-1:1
        result = BigInt(result^2 * number^power[i]) % base;
    end;

    return result;
end
function st(n::Integer)
    for s=1:BigInt(floor(log2(n)))
        t, r = divrem(n, 2^s);
        r == 0 && isodd(t) && return (s, t);
    end
    
    throw(error("Wrong number pattern"));
end
"""
    witness(a, n)

Test for evidence of simplicity.
"""
function witness(a::Integer, n::Integer)
    if gcd(a, n) > 1
        return false;
    end
    
    s, t = st(n-1);    
    b = pow(BigInt(a), t, n);
    
    if b == 1 || b == n-1
        return true;
    end
    
    for i=1:s-1
        b = pow(b, 2, n);
        if b == n-1
            return true;
        end
    end
    
    return false;
end
"""
    mrpt(n, k)

Miller–Rabin primality test
"""
function mrpt(n::Signed, k::Int=1)
    for i=1:k
        a = rand(2:n-1);
        if !witness(a, n)
            return false;
        end
    end
    
    return true;
end
"""
    primegen(b)

Generates b-bit prime number using Miller–Rabin primality test
"""
function primegen(b::Signed)::BigInt
    while true 
        p = BigInt(rand(2^(b-1):2^b));
        p |= 1; # установка младшего бита
        
        try
            mrpt(p) && return p;
        catch
            continue;
        end
    end
end    

primegen

## Задание 1.
Написать функцию генерации ключей GenerationKey[ L ], которая возвращает ключи PublicKey, PrinateKey криптосистемы RSA, где n = p q, простые числа p и q длины L бит генерируются при помощи изученного нами ранее алгоритма Миллера-Рабина или теоремы Диемитко. 

In [2]:
generationKey(L::Int) = let p=primegen(L), q=primegen(L)
    n = p * q;
    ϕ = (p-1) * (q-1);
    
    e = rand(2:ϕ-1);
    while gcd(e, ϕ) != 1
        e = rand(2:ϕ-1);
    end
    return (e, n), invmod(e, ϕ)
end

generationKey (generic function with 1 method)

In [3]:
@time public, private = generationKey(50)

  0.002072 seconds (46.12 k allocations: 923.070 KiB)


((900112755833617010941853436587, 1098989568807374265250848183661), 174527295127986384768233776523)

## Задание 2.
Задать функцию EnRSA[m_Integer, PublicKey_], которая шифрует число m, 0 < m < n, при помощи открытого ключа PublicKey = {e,n}.

In [4]:
enRSA(m::Integer, public::Tuple) = pow(m, public[1], public[2])

enRSA (generic function with 1 method)

## Задание 3.
Определить дешифрующую функцию DeRSA[c_Integer, PublicKey_, PrivateKey_], возвращающую число m.

In [5]:
deRSA(c::Integer, public::Tuple, private::Integer) = pow(c, private, public[2])

deRSA (generic function with 1 method)

### Тест

In [6]:
m = rand(2:public[2]-1)

1056195762496137223226894589374

In [7]:
@time c = enRSA(m, public)

  0.001809 seconds (4.13 k allocations: 166.604 KiB)


60321828745308350250158507030

In [8]:
@time M = deRSA(c, public, private)

  0.002172 seconds (3.34 k allocations: 122.835 KiB)


1056195762496137223226894589374

### Работает!

In [9]:
e = 65537
digits(e, base=2) .|> string |> join

"10000000000000001"

In [10]:
d = 9054851157546901446407736431037005394599645174440125709528488185124729724246443297544993312731942938306657102136973353762949755696434479496090572659899440190453603132012222980690263963141408862803615400770040598521709306263314491947019405782432281947588353402223301916956406336346397075978289270501050468994119868614290524781489264467106920616175672621995272742662486917988264000240414115031968723852799594669873393589423641966431446926067991740423405123044093886933240232395613164314933592639917833772516482654578891125886569880148636359939966997512790475166557863599177793672220598894381406545596151147206439711153

9054851157546901446407736431037005394599645174440125709528488185124729724246443297544993312731942938306657102136973353762949755696434479496090572659899440190453603132012222980690263963141408862803615400770040598521709306263314491947019405782432281947588353402223301916956406336346397075978289270501050468994119868614290524781489264467106920616175672621995272742662486917988264000240414115031968723852799594669873393589423641966431446926067991740423405123044093886933240232395613164314933592639917833772516482654578891125886569880148636359939966997512790475166557863599177793672220598894381406545596151147206439711153

In [11]:
n = 11148370849373497653451508970146012071122993533670533883625183734520372194964102092639605987911203162648945829471178333375210184840827099055707079849931046623365729634842852855260150842615038749606623004325871702147609671324043619363738696163127267743736575557420787858943678415651584166229365844839889997867347181675015424770694878536217111691832047850689019359464171435807185864527152428747606943123764746549087557639197708964779103488547840013637158498454472874160339328858354757021680672964648778854793991227820696514039780900621673114778340908536909733075084403336628504799400480421071417009796343636060725399691

11148370849373497653451508970146012071122993533670533883625183734520372194964102092639605987911203162648945829471178333375210184840827099055707079849931046623365729634842852855260150842615038749606623004325871702147609671324043619363738696163127267743736575557420787858943678415651584166229365844839889997867347181675015424770694878536217111691832047850689019359464171435807185864527152428747606943123764746549087557639197708964779103488547840013637158498454472874160339328858354757021680672964648778854793991227820696514039780900621673114778340908536909733075084403336628504799400480421071417009796343636060725399691

In [12]:
function factorN(e::Integer, n::Integer, d::Integer)
    s, t = st(e * d - 1);
    
    while true
        a = rand(2:n-2);
        if gcd(a, n) > 1
            p = gcd(a, n);
            q = n / p;
            return (p, q);
        end;
        
        u = pow(a, t, n);
        v = pow(u, 2, n);
        
        u == 1 && continue;
        
        while v != 1
            u = v;
            v = pow(u, 2, n);
        end;
        
        u%n == n-1 && continue;
        
        return (gcd(u+1, n), gcd(u-1, n))
    end
end

factorN (generic function with 1 method)

In [13]:
@time p, q = factorN(e, n, d)

  0.036434 seconds (86.08 k allocations: 5.975 MiB)


(100103549190573633808776187512152445545530578886521005807536385846409064350292076056894756229169360411757135169369964830366483657953460385875160656041055783229921482985948281520640897872188795079945613538753552715827228797765798265715346839394569697990724376925924827347709664905418254457840241795900937582653, 111368387429996306819186598549290329842687539927537058599873728446652100125084664979832339907164857565481979943619642996820702919196333888665084418612162515165928552043526145805232845525631220446848890111890048406192485782448187809397662167658294395952466306577915492896058251364966064284461550689784652755047)

In [14]:
n == p * q

true